In [11]:
import numpy as np
import pandas as pd

data = pd.read_csv('games.csv', header=0, delimiter=',')
data = data.drop(columns=['creationTime', 'seasonId', 't1_champ1_sum1', 't1_champ1_sum2', 't1_champ2_sum1', 't1_champ2_sum2','t1_champ3_sum1', 't1_champ3_sum2',
                 't1_champ4_sum1', 't1_champ4_sum2', 't1_champ5_sum1', 't1_champ5_sum2', 't2_champ1_sum1', 't2_champ1_sum2', 't2_champ2_sum1', 't2_champ2_sum2','t2_champ3_sum1', 't2_champ3_sum2',
                 't2_champ4_sum1', 't2_champ4_sum2', 't2_champ5_sum1', 't2_champ5_sum2', 't1_towerKills', 't1_inhibitorKills', 't1_baronKills', 't1_dragonKills', 't1_riftHeraldKills',
                't2_towerKills', 't2_inhibitorKills', 't2_baronKills', 't2_dragonKills', 't2_riftHeraldKills'])
msk = np.random.rand(len(data)) < 0.9
train_x = data[msk]
test_x = data[~msk]
train_x.head()

,gameId,gameDuration,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_champ1id,...,t2_champ1id,t2_champ2id,t2_champ3id,t2_champ4id,t2_champ5id,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
0,3326086514,1949,1,2,1,1,1,1,2,8,...,104,498,122,238,412,114,67,43,16,51
1,3229566029,1851,1,1,1,1,0,1,1,119,...,54,25,120,157,92,11,67,238,51,420
2,3327363504,1493,1,2,1,1,1,2,0,18,...,69,412,126,24,22,157,238,121,57,28
3,3326856598,1758,1,1,1,1,1,1,0,57,...,90,19,412,92,22,164,18,141,40,51
4,3330080762,2094,1,2,1,1,1,1,0,19,...,37,59,141,38,51,86,11,201,122,18
